In [112]:
import pandas as pd
import os

In [113]:
def read_csv(path: str) -> pd.DataFrame:
    root_path = "../data/raw"
    full_path = root_path + "/" +  path
    df = pd.read_csv(full_path)
    return df

<h1>Create Linking Tables for All Regional Data</h1>
Obtain dfs in which every unique zone, state, and lga are represented.
<ol>
    <li>There are 6 unique geopolitical zones</li>
    <li>There are 36 unique states, plus Federal Capital Territory, so potentially 37</li>
    <li>there are 774 unique LGAs</li>
</ol>

In [114]:
# Open a file, and identify whether or not every zone, state, and lga is represented
# In this one, we can see that all zones and states are represented, but not all lgas. We will use it to create the linking table of zone to state.
path = "p2r6_sect_a_2_5_6_8_11b_12.csv"
df = read_csv(path=path)
df = df[["zone", "state", "lga", "sector"]]
df.nunique()

zone        6
state      37
lga       386
sector      2
dtype: int64

In [115]:
# ZONE TO STATE
# Check to make sure there is no overlapping data; that the relationship of zone to state is one-to-many
# Beecause our grouped dataframe below is empty, we can be sure this is the case and can continue to the next step
grouped = df.groupby(by="state").agg(unique_zone=("zone", "nunique"))
grouped[grouped["unique_zone"] > 1]

,unique_zone
state,


In [116]:
# Create the linking table of zone to state
zone_state_link = df[["zone", "state"]].drop_duplicates(keep="first")
print(len(df))
print(len(zone_state_link))
print(sorted(zone_state_link["zone"].unique()))
print(sorted(zone_state_link["state"].unique()))
zone_state_link.rename(columns={"state": "state_id", "zone":"zone_id"}, inplace=True)
zone_state_link.set_index(["state_id"], inplace=True)
zone_state_link.to_csv("../data/processed/linking_tables/zone_to_state.csv")
zone_state_link

2799
37
[1, 2, 3, 4, 5, 6]
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37]


,zone_id
state_id,
1,4
2,2
34,2
3,5
12,5
4,4
24,6
5,2
31,1


In [117]:
# STATE TO LGA
# Combine geographical data from every file to get the full dataset of lgas. Later, we will check for duplicates and incorrect data. Check the unique values in the dataset. We can see that we only have 400 lgas represented in this dataset.
path_to_files = "../data/raw"
csvs = os.listdir(path_to_files)
dfs = []
for file_name in csvs:
    df = pd.read_csv(path_to_files + "/" + file_name)
    df = df[["state", "lga"]]
    dfs.append(df)

region_df = pd.concat(dfs)
region_df.nunique()


/var/folders/lw/jy_c2z0x221fsb78kk1_h2800000gn/T/ipykernel_51275/742062374.py:7: DtypeWarning: Columns (10,17) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(path_to_files + "/" + file_name)
/var/folders/lw/jy_c2z0x221fsb78kk1_h2800000gn/T/ipykernel_51275/742062374.py:7: DtypeWarning: Columns (16,36,82) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(path_to_files + "/" + file_name)
/var/folders/lw/jy_c2z0x221fsb78kk1_h2800000gn/T/ipykernel_51275/742062374.py:7: DtypeWarning: Columns (28,33) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(path_to_files + "/" + file_name)


state     37
lga      400
dtype: int64

In [118]:
# Check to make sure that states to lgas have a 1-to-many relationship. The dataframe below is not empty, indicating two user entry errors.
grouped = region_df.groupby(by="lga").agg(unique_state=("state", "nunique"))
grouped[grouped["unique_state"] > 1]


,unique_state
lga,
209,2
3215,2


In [119]:
# Find any data which erroneously associates lga 209 to the wrong state
region_df[region_df["lga"] == 209]
# Many interviewers made the same mistake, it seems. LGA 209 is known as Lamurde, and this is located in Adamawa state, which the correct encoding for is 2, not 34.

,state,lga
544,2,209
545,2,209
546,2,209
547,2,209
548,2,209
...,...,...
176,34,209
177,2,209
219,2,209
220,2,209


In [120]:
# Fix the incorrect lga 209 data based on the findings above
region_df.loc[region_df['lga'] == 209, 'state'] = 2
region_df[region_df["lga"] == 209]

,state,lga
544,2,209
545,2,209
546,2,209
547,2,209
548,2,209
...,...,...
176,2,209
177,2,209
219,2,209
220,2,209


In [121]:
# Find any data which erroneously associates lga 3215 to the wrong state
# LGA 3215 is OBIO AKPOR. This is located in the Rivers State, which is encoded as 32. There are many entry errors which place it in state 6 instead. 
region_df[region_df["lga"] == 3215]

,state,lga
8858,32,3215
8859,32,3215
8860,32,3215
8861,32,3215
8882,32,3215
...,...,...
2562,32,3215
2563,32,3215
2564,32,3215
2568,32,3215


In [122]:
# Fix the incorrect lga 3215 data based on the findings above
region_df.loc[region_df['lga'] == 3215, 'state'] = 32
region_df[region_df["lga"] == 3215]

,state,lga
8858,32,3215
8859,32,3215
8860,32,3215
8861,32,3215
8882,32,3215
...,...,...
2562,32,3215
2563,32,3215
2564,32,3215
2568,32,3215


In [123]:
# Create the linking table of state to LGA

state_lga_link = region_df[["lga", "state"]].drop_duplicates(keep="first")
print(len(df))
print(len(state_lga_link))
print(sorted(state_lga_link["state"].unique()))
print(sorted(state_lga_link["lga"].unique()))
state_lga_link.rename(columns={"state": "state_id", "lga":"lga_id"}, inplace=True)
state_lga_link.set_index(["lga_id"], inplace=True)
#state_lga_link.to_csv("../data/processed/linking_tables/state_to_lga.csv")
state_lga_link

2824
400
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37]
[102, 104, 105, 107, 108, 109, 110, 111, 112, 113, 115, 117, 201, 203, 204, 205, 207, 208, 209, 210, 211, 212, 213, 215, 218, 219, 220, 221, 301, 302, 304, 305, 307, 308, 309, 312, 313, 316, 317, 318, 320, 322, 323, 326, 327, 329, 331, 402, 405, 406, 407, 408, 409, 412, 416, 417, 419, 420, 421, 502, 503, 504, 505, 506, 507, 508, 509, 511, 513, 514, 516, 517, 518, 520, 601, 602, 604, 605, 606, 607, 608, 701, 702, 704, 705, 708, 709, 711, 713, 715, 716, 717, 718, 719, 721, 723, 805, 821, 827, 901, 905, 908, 909, 910, 913, 915, 917, 918, 1002, 1006, 1007, 1008, 1010, 1011, 1012, 1013, 1014, 1015, 1018, 1019, 1022, 1023, 1025, 1101, 1102, 1104, 1105, 1106, 1109, 1112, 1113, 1201, 1202, 1203, 1207, 1208, 1209, 1211, 1212, 1214, 1215, 1216, 1218, 1301, 1303, 1306, 1307, 1308, 1309, 1310, 1311, 1312, 1315, 1316, 1401, 1402, 1403, 1404, 1405, 1406

,state_id
lga_id,
115,1
105,1
111,1
102,1
107,1
...,...
1010,10
2901,29
3305,33
